# LLM 응답 객체 생성
- API key 로딩

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ["OPENAI_API_KEY"]

from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-4o", api_key=api_key)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=api_key)

In [5]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

# 응답 객체 생성

In [6]:
# BaseModel 상속 받기
class EmailParser(BaseModel):
    sender_name: str = Field(description="이메일 송신자명")
    title: str = Field(description="이메일 제목")
    content: str = Field(description="이메일 내용")

# outputparser 객체 생성

In [7]:
parser = JsonOutputParser(pydantic_object=EmailParser)
parser

JsonOutputParser(pydantic_object=<class '__main__.EmailParser'>)

# format 명령 확인

In [8]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"sender_name": {"description": "이메일 송신자명", "title": "Sender Name", "type": "string"}, "title": {"description": "이메일 제목", "title": "Title", "type": "string"}, "content": {"description": "이메일 내용", "title": "Content", "type": "string"}}, "required": ["sender_name", "title", "content"]}
```


# 템플릿 객체 생성

In [10]:
prompt_template = PromptTemplate(
    template="이메일을 읽고 요약해 주세요.\n{format_instructions}\n{email}\n",
    input_variables=["email"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt_template

PromptTemplate(input_variables=['email'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"sender_name": {"description": "이메일 송신자명", "title": "Sender Name", "type": "string"}, "title": {"description": "이메일 제목", "title": "Title", "type": "string"}, "content": {"description": "이메일 내용", "title": "Content", "type": "string"}}, "required": ["sender_name", "title", "content"]}\n```'}, template='이메일을 읽고 요약해 주세요.\n{format_instructions}\n{email}\n')

In [11]:
# email 매개변수 넘김
prompt = prompt_template.format(email="보내는 사람: 김철수\n제목: 오늘의 회의록\n내용: 오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요.")
print(prompt)

이메일을 읽고 요약해 주세요.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"sender_name": {"description": "이메일 송신자명", "title": "Sender Name", "type": "string"}, "title": {"description": "이메일 제목", "title": "Title", "type": "string"}, "content": {"description": "이메일 내용", "title": "Content", "type": "string"}}, "required": ["sender_name", "title", "content"]}
```
보내는 사람: 김철수
제목: 오늘의 회의록
내용: 오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요.



# 템플릿 실행 요청

In [12]:
# llm에 prompt 보내기
response = llm.invoke(prompt)

# 응답 받은 결과를 parser로 파싱하기
parsed_output = parser.invoke(response)
print(parsed_output)

{'sender_name': '김철수', 'title': '오늘의 회의록', 'content': '오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요.'}


# LCEL로 실행

- LCEL(LangChain Expression Language)

In [13]:
# chain = 프롬프트템플릿 | llm_모델 | 파서 
# chain.invoke({"변수": "질문"})
chain = prompt_template | llm | parser

email = "보내는 사람: 김철수\n제목: 오늘의 회의록\n내용: 오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요."

response = chain.invoke({"email":email})

In [14]:
response

{'sender_name': '김철수',
 'title': '오늘의 회의록',
 'content': '오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요.'}